In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

In [ ]:
# Collect historical data for the assets in the portfolio using yahoo finance
assets = ["PFE", "JNJ", "GOOGL", "AAPL", "COST", "WMT", "KR", "JPM",
          "BAC", "HSBC"]
start_date = "2018-01-01"
end_date = "2023-04-01"
data = yf.download(assets, start=start_date, end=end_date)

[*********************100%***********************]  10 of 10 completed


In [ ]:
data

Adj Close                                                \
                  AAPL        BAC        COST       GOOGL       HSBC   
Date                                                                   
2018-01-02   40.831585  26.421577  175.129333   53.660500  39.990192   
2018-01-03   40.824482  26.333210  177.231033   54.576000  39.791801   
2018-01-04   41.014111  26.677839  175.854721   54.787998  39.875736   
2018-01-05   41.481064  26.801552  174.599258   55.514500  39.807060   
2018-01-08   41.327000  26.615982  175.278137   55.710499  39.486595   
...                ...        ...         ...         ...        ...   
2023-03-27  158.061371  28.264458  492.193420  102.459999  33.198647   
2023-03-28  157.432236  27.897388  485.756836  101.029999  33.021008   
2023-03-29  160.547943  28.443033  489.848297  101.389999  33.771038   
2023-03-30  162.135742  28.075962  490.457062  100.889999  33.840118   
2023-03-31  164.672226  28.373589  495.835815  103.730003  33.682217   

                                                                      ...  \
                   JNJ         JPM         KR        PFE         WMT  ...   
Date                                                                  ...   
2018-01-02  120.209435   91.977722  25.253582  28.254051   89.268608  ...   
2018-01-03  121.357758   92.071472  24.852024  28.463390   90.047295  ...   
2018-01-04  121.349144   93.390457  24.075674  28.525425   90.128784  ...   
2018-01-05  122.350662   92.790894  24.379076  28.579704   90.663025  ...   
2018-01-08  122.506050   92.927933  24.959105  28.261801   92.003075  ...   
...                ...         ...        ...        ...         ...  ...   
2023-03-27  152.152023  127.489464  48.372807  39.789448  143.624405  ...   
2023-03-28  150.683105  127.876427  48.661266  39.561913  143.066528  ...   
2023-03-29  152.161942  128.134399  48.780636  39.819126  143.684174  ...   
2023-03-30  152.281036  127.747437  48.800529  39.947735  145.118729  ...   
2023-03-31  153.839279  129.295288  49.108891  40.363239  146.891998  ...   

               Volume                                                  \
                 AAPL        BAC     COST     GOOGL     HSBC      JNJ   
Date                                                                    
2018-01-02  102223600   57121600  2509300  31766000  2119600  6842100   
2018-01-03  118071600   57865700  3364400  31318000  2283600  5350500   
2018-01-04   89738400   76512500  3525800  26052000  2857400  4837600   
2018-01-05   94640000   56445200  2819000  30250000  1871100  6158800   
2018-01-08   82271200   42914800  1961800  24644000  1255300  5129800   
...               ...        ...      ...       ...      ...      ...   
2023-03-27   52390300  102469800  1452100  31120900  2071800  6772300   
2023-03-28   45992200   63416500  1407800  32057900  1620200  6115300   
2023-03-29   51305700   62666400  1643000  28779600  2183500  6012000   
2023-03-30   49501700   67427100  1545500  33086200  1263000  4555700   
2023-03-31   68749800   56482100  2082000  36863400  1716500  9890000   

                                                    
                 JPM        KR       PFE       WMT  
Date                                                
2018-01-02  13578800  11693300  17059833  10150500  
2018-01-03  11901000  10788400  14183151   8903600  
2018-01-04  12953700  12357600  13046517   6830400  
2018-01-05  14155000   9629900  13167517   7284000  
2018-01-08  12466500  12321700  19964341   8843900  
...              ...       ...       ...       ...  
2023-03-27  18853900   6508200  20262800   8925200  
2023-03-28  10820500   4732500  13537500   7094300  
2023-03-29  14595300   5164600  14429500   8072300  
2023-03-30  12134700   3743800  12634500   7293000  
2023-03-31  13105200   4811100  23393600   6956000  

[1321 rows x 60 columns]

In [ ]:
data = data["Adj Close"]
data.dropna(inplace=True)
returns = data.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()

<ipython-input-4-8078115c3e1b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [ ]:
returns

,AAPL,BAC,COST,GOOGL,HSBC,JNJ,JPM,KR,PFE,WMT
Date,,,,,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,-0.000174,-0.003345,0.012001,0.017061,-0.004961,0.009553,0.001019,-0.015901,0.007409,0.008723
2018-01-04,0.004645,0.013087,-0.007766,0.003884,0.002109,-0.000071,0.014326,-0.031239,0.002179,0.000905
2018-01-05,0.011385,0.004637,-0.007139,0.013260,-0.001722,0.008253,-0.006420,0.012602,0.001903,0.005928
2018-01-08,-0.003714,-0.006924,0.003888,0.003531,-0.008050,0.001270,0.001477,0.023792,-0.011123,0.014781
...,...,...,...,...,...,...,...,...,...,...
2023-03-27,-0.012293,0.049742,-0.004139,-0.028263,0.016929,0.004258,0.028661,-0.008563,-0.004209,0.016714
2023-03-28,-0.003980,-0.012987,-0.013077,-0.013957,-0.005351,-0.009654,0.003035,0.005963,-0.005718,-0.003884
2023-03-29,0.019791,0.019559,0.008423,0.003563,0.022714,0.009814,0.002017,0.002453,0.006502,0.004317


In [ ]:
mean_returns

AAPL     0.001275
BAC      0.000300
COST     0.000905
GOOGL    0.000700
HSBC     0.000027
JNJ      0.000272
JPM      0.000459
KR       0.000698
PFE      0.000404
WMT      0.000484
dtype: float64

In [ ]:
data

,AAPL,BAC,COST,GOOGL,HSBC,JNJ,JPM,KR,PFE,WMT
Date,,,,,,,,,,
2018-01-02,40.831585,26.421577,175.129333,53.660500,39.990192,120.209435,91.977722,25.253582,28.254051,89.268608
2018-01-03,40.824482,26.333210,177.231033,54.576000,39.791801,121.357758,92.071472,24.852024,28.463390,90.047295
2018-01-04,41.014111,26.677839,175.854721,54.787998,39.875736,121.349144,93.390457,24.075674,28.525425,90.128784
2018-01-05,41.481064,26.801552,174.599258,55.514500,39.807060,122.350662,92.790894,24.379076,28.579704,90.663025
2018-01-08,41.327000,26.615982,175.278137,55.710499,39.486595,122.506050,92.927933,24.959105,28.261801,92.003075
...,...,...,...,...,...,...,...,...,...,...
2023-03-27,158.061371,28.264458,492.193420,102.459999,33.198647,152.152023,127.489464,48.372807,39.789448,143.624405
2023-03-28,157.432236,27.897388,485.756836,101.029999,33.021008,150.683105,127.876427,48.661266,39.561913,143.066528
2023-03-29,160.547943,28.443033,489.848297,101.389999,33.771038,152.161942,128.134399,48.780636,39.819126,143.684174


In [ ]:
# Train an LSTM model on the preprocessed data to predict future asset returns
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data = scaled_data[0:train_size, :]
test_data = scaled_data[train_size:len(scaled_data), :]
X_train, y_train = [], []
for i in range(60, train_size):
    X_train.append(train_data[i-60:i, :])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))


In [ ]:
scaled_data

array([[0.04528403, 0.31392347, 0.02068617, ..., 0.14957717, 0.12529654,
        0.16915463],
       [0.04523545, 0.31105617, 0.02550088, ..., 0.13977285, 0.13142053,
        0.17873998],
       [0.04653229, 0.32223849, 0.02234794, ..., 0.12081771, 0.13323528,
        0.17974309],
       ...,
       [0.86400099, 0.37951439, 0.74166417, ..., 0.7240068 , 0.46362008,
        0.83899034],
       [0.87485964, 0.3676039 , 0.74305877, ..., 0.72449251, 0.46738238,
        0.85664919],
       [0.89220616, 0.37726109, 0.75538075, ..., 0.73202137, 0.47953751,
        0.87847749]])

In [ ]:
y_train

array([[0.03913071, 0.31973052, 0.02199471, ..., 0.056913  , 0.11120608,
        0.06784442],
       [0.03734032, 0.30015997, 0.00986162, ..., 0.04246912, 0.10113332,
        0.02949844],
       [0.04012354, 0.30821846, 0.00956265, ..., 0.04925328, 0.11555555,
        0.04351376],
       ...,
       [0.87218823, 0.74560245, 0.81746043, ..., 0.84942201, 0.6631022 ,
        0.74715569],
       [0.84211631, 0.73398141, 0.82897137, ..., 0.89105084, 0.67569623,
        0.78527187],
       [0.81638897, 0.72330268, 0.81693833, ..., 0.86250527, 0.70564201,
        0.77853834]])

In [ ]:
X_train

array([[[0.04528403, 0.31392347, 0.02068617, ..., 0.14957717,
         0.12529654, 0.16915463],
        [0.04523545, 0.31105617, 0.02550088, ..., 0.13977285,
         0.13142053, 0.17873998],
        [0.04653229, 0.32223849, 0.02234794, ..., 0.12081771,
         0.13323528, 0.17974309],
        ...,
        [0.04725268, 0.33268154, 0.01941008, ..., 0.05538115,
         0.1009046 , 0.05136205],
        [0.04004222, 0.30620387, 0.01071612, ..., 0.04662715,
         0.10021768, 0.03510451],
        [0.03701474, 0.30246253, 0.01169869, ..., 0.04990986,
         0.10685662, 0.05438959]],

       [[0.04523545, 0.31105617, 0.02550088, ..., 0.13977285,
         0.13142053, 0.17873998],
        [0.04653229, 0.32223849, 0.02234794, ..., 0.12081771,
         0.13323528, 0.17974309],
        [0.04972569, 0.32625263, 0.01947185, ..., 0.12822549,
         0.13482316, 0.18631939],
        ...,
        [0.04004222, 0.30620387, 0.01071612, ..., 0.04662715,
         0.10021768, 0.03510451],
        [0.0

In [ ]:
X_train.shape

(996, 60, 10)

In [ ]:
y_train.shape

(996, 10)

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=25, return_sequences=True))
model.add(LSTM(units=10))
model.add(Dense(units=len(assets)))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
32/32 [==============================] - 12s 148ms/step - loss: 0.0831
Epoch 2/50
32/32 [==============================] - 4s 111ms/step - loss: 0.0186
Epoch 3/50
32/32 [==============================] - 3s 109ms/step - loss: 0.0099
Epoch 4/50
32/32 [==============================] - 4s 130ms/step - loss: 0.0071
Epoch 5/50
32/32 [==============================] - 4s 126ms/step - loss: 0.0057
Epoch 6/50
32/32 [==============================] - 4s 111ms/step - loss: 0.0048
Epoch 7/50
32/32 [==============================] - 4s 111ms/step - loss: 0.0043
Epoch 8/50
32/32 [==============================] - 5s 149ms/step - loss: 0.0038
Epoch 9/50
32/32 [==============================] - 4s 110ms/step - loss: 0.0035
Epoch 10/50
32/32 [==============================] - 3s 109ms/step - loss: 0.0033
Epoch 11/50
32/32 [==============================] - 5s 146ms/step - loss: 0.0033
Epoch 12/50
32/32 [==============================] - 3s 108ms/step - loss: 0.0029
Epoch 13/50
32/32 [=====

In [ ]:
# Define the objective function for the portfolio optimization
def objective(x,mean_returns,cov_matrix):
    port_returns = np.sum(mean_returns * x)
    port_volatility = np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))
    penalty = 10 * np.abs(np.sum(x) - 1) # penalty for sum of portfolio weights not equal to 1
    return -port_returns/port_volatility + penalty

In [ ]:
# Define the ABC optimization algorithm
def abc_optimization(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix):
    n_dim = len(lb)
    best_sol = None
    best_obj = np.inf
    solutions = np.random.dirichlet(np.ones(len(lb)), size=n_solutions) * (ub-lb) + lb
    solutions = solutions / solutions.sum(axis=1, keepdims=True)
    for i in range(n_solutions):
        sol = np.random.uniform(lb, ub, n_dim)
        obj = objective_func(sol,mean_returns,cov_matrix)
        if obj < best_obj:
            best_sol = sol
            best_obj = obj
        for j in range(n_iter):
            partner = np.random.choice(list(set(range(n_solutions)) - set([i])))
            phi = np.random.uniform(-1, 1, n_dim)
            new_sol = sol + phi * (sol - solutions[partner])
            new_sol = np.maximum(new_sol, lb)
            new_sol = np.minimum(new_sol, ub)
            new_obj = objective_func(new_sol,mean_returns,cov_matrix)
            if new_obj < obj:
                sol = new_sol
                obj = new_obj
                if obj < best_obj:
                    best_sol = sol
                    best_obj = obj
    return best_sol, best_obj

In [ ]:
def fa_algorithm(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix, alpha=0.5, beta=1, gamma=1):
    n_dim = len(lb)
    best_sol = None
    best_obj = np.inf
    solutions = np.random.dirichlet(np.ones(len(lb)), size=n_solutions) * (ub-lb) + lb
    solutions = solutions / solutions.sum(axis=1, keepdims=True)
    for i in range(n_iter):
        attractiveness = np.zeros(n_solutions)
        for j in range(n_solutions):
            for k in range(n_solutions):
                if objective_func(solutions[k],mean_returns,cov_matrix) < objective_func(solutions[j],mean_returns,cov_matrix):
                    attractiveness[j] += (1 / (1 + np.linalg.norm(solutions[j] - solutions[k])))
            attractiveness[j] *= np.random.rand()
        index = np.argsort(attractiveness)[::-1]
        best_firefly = solutions[index[0]]
        for j in range(n_solutions):
            beta0 = 1
            beta = beta0 * np.exp(-0.2 * j)
            r = np.linalg.norm(solutions[j] - best_firefly)
            new_sol = solutions[j] + beta * (best_firefly - solutions[j]) + 0.01 * np.random.randn(n_dim)
            new_sol = np.maximum(new_sol, lb)
            new_sol = np.minimum(new_sol, ub)
            new_obj = objective_func(new_sol,mean_returns,cov_matrix)
            if new_obj < objective_func(solutions[j],mean_returns,cov_matrix):
                solutions[j] = new_sol
            if new_obj < best_obj:
                best_sol = new_sol
                best_obj = new_obj
    return best_sol, best_obj


In [ ]:
# Define the hybrid algorithm
def hybrid_algorithm(n_solutions, n_iter, lb, ub, objective_func, alpha, beta0, gamma,mean_returns,cov_matrix):
    abc_solutions, abc_obj = abc_optimization(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix)
    fa_solutions, fa_obj = fa_algorithm(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix)
    hybrid_solutions = alpha * abc_solutions + (1 - alpha) * fa_solutions
    hybrid_solutions = np.maximum(hybrid_solutions, lb)
    hybrid_solutions = np.minimum(hybrid_solutions, ub)
    hybrid_obj = objective_func(hybrid_solutions,mean_returns,cov_matrix)
    return hybrid_solutions, hybrid_obj


In [ ]:
# Make predictions for all assets in test data
test_inputs = []
for i in range(60, len(test_data)):
    test_inputs.append(test_data[i-60:i, :])
test_inputs = np.array(test_inputs)
test_predictions = model.predict(test_inputs)


7/7 [==============================] - 2s 33ms/step


In [ ]:
test_inputs.shape

(205, 60, 10)

In [ ]:
test_predictions.shape

(205, 10)

In [ ]:
test_predictions

array([[0.69964164, 0.4610075 , 0.5876236 , ..., 0.38533273, 0.25210753,
        0.53529054],
       [0.70613277, 0.4718561 , 0.58684963, ..., 0.39930075, 0.25558472,
        0.5372778 ],
       [0.7108259 , 0.48037562, 0.5839187 , ..., 0.4106836 , 0.25544584,
        0.53719515],
       ...,
       [0.59672445, 0.5899946 , 0.49494174, ..., 0.34896418, 0.31436417,
        0.6601959 ],
       [0.6076583 , 0.59228504, 0.50337243, ..., 0.35920414, 0.32559094,
        0.6671366 ],
       [0.6178331 , 0.59544116, 0.51166147, ..., 0.36918604, 0.33661917,
        0.67377883]], dtype=float32)

In [ ]:
# Inverse transform the predictions using the fitted scaler for each asset
test_predictions_for_all_assets = scaler.inverse_transform(test_predictions)
test_predictions_for_all_assets = test_predictions.reshape((len(test_predictions), len(assets)))

# Construct a DataFrame with the predicted returns for each asset
test_df = pd.DataFrame(test_predictions_for_all_assets, columns=assets)
returns_df = test_df.pct_change().dropna()
cov_matrix_df = returns_df.cov().dropna()

In [ ]:
test_df.shape

(205, 10)

In [ ]:
returns_df.shape

(204, 10)

In [ ]:
cov_matrix_df.shape

(10, 10)

In [ ]:
returns_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
1,0.009278,0.023532,-0.001317,-0.002487,0.109745,0.009333,-0.000589,0.036249,0.013793,0.003713
2,0.006646,0.018055,-0.004994,-0.005049,0.091762,0.007514,-0.005134,0.028507,-0.000543,-0.000154
3,0.009683,0.018818,-0.007247,-0.007777,0.097707,0.007716,0.003025,0.029644,0.021876,0.009885
4,0.007665,0.012438,-0.009241,-0.009711,0.073133,0.005877,-0.000371,0.022066,0.012102,0.006415
5,0.007187,0.008417,-0.009631,-0.010794,0.057347,0.004635,0.000343,0.017417,0.013653,0.007042
...,...,...,...,...,...,...,...,...,...,...
200,0.019016,-0.002627,0.015054,0.011906,-0.013947,0.014308,0.001806,0.025987,0.032756,0.008312
201,0.019883,-0.000072,0.016500,0.013944,-0.012368,0.015091,0.002858,0.028664,0.035819,0.009934
202,0.019210,0.001887,0.016783,0.014978,-0.010141,0.014775,0.003375,0.029270,0.035930,0.010328
203,0.018323,0.003882,0.017034,0.015898,-0.007939,0.014268,0.003861,0.029344,0.035713,0.010513


In [ ]:
cov_matrix_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
PFE,0.000083,0.000062,0.000031,0.000023,0.000092,0.000053,0.000036,0.000141,0.000179,0.000058
JNJ,0.000062,0.000136,0.000034,0.000037,0.000239,0.000043,0.000058,0.000155,0.000180,0.000081
GOOGL,0.000031,0.000034,0.000059,0.000052,-0.000011,0.000029,0.000017,0.000070,0.000097,0.000032
AAPL,0.000023,0.000037,0.000052,0.000054,-0.000001,0.000021,0.000017,0.000057,0.000081,0.000028
COST,0.000092,0.000239,-0.000011,-0.000001,0.000641,0.000063,0.000086,0.000265,0.000248,0.000120
WMT,0.000053,0.000043,0.000029,0.000021,0.000063,0.000039,0.000021,0.000100,0.000121,0.000036
KR,0.000036,0.000058,0.000017,0.000017,0.000086,0.000021,0.000039,0.000066,0.000103,0.000047
JPM,0.000141,0.000155,0.000070,0.000057,0.000265,0.000100,0.000066,0.000285,0.000331,0.000112
BAC,0.000179,0.000180,0.000097,0.000081,0.000248,0.000121,0.000103,0.000331,0.000467,0.000158
HSBC,0.000058,0.000081,0.000032,0.000028,0.000120,0.000036,0.000047,0.000112,0.000158,0.000069


In [ ]:
roi=[]
s_r=[]

In [ ]:
n_solutions = 100
n_iter = 50
alpha = 0.1
beta0 = 1
gamma = 2

lb = np.array([0] * len(assets))
ub = np.array([1] * len(assets))

In [ ]:
lb

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
ub

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
returns_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
1,0.009278,0.023532,-0.001317,-0.002487,0.109745,0.009333,-0.000589,0.036249,0.013793,0.003713
2,0.006646,0.018055,-0.004994,-0.005049,0.091762,0.007514,-0.005134,0.028507,-0.000543,-0.000154
3,0.009683,0.018818,-0.007247,-0.007777,0.097707,0.007716,0.003025,0.029644,0.021876,0.009885
4,0.007665,0.012438,-0.009241,-0.009711,0.073133,0.005877,-0.000371,0.022066,0.012102,0.006415
5,0.007187,0.008417,-0.009631,-0.010794,0.057347,0.004635,0.000343,0.017417,0.013653,0.007042
...,...,...,...,...,...,...,...,...,...,...
200,0.019016,-0.002627,0.015054,0.011906,-0.013947,0.014308,0.001806,0.025987,0.032756,0.008312
201,0.019883,-0.000072,0.016500,0.013944,-0.012368,0.015091,0.002858,0.028664,0.035819,0.009934
202,0.019210,0.001887,0.016783,0.014978,-0.010141,0.014775,0.003375,0.029270,0.035930,0.010328
203,0.018323,0.003882,0.017034,0.015898,-0.007939,0.014268,0.003861,0.029344,0.035713,0.010513


In [ ]:
returns_df.shape[0]

204

In [ ]:
def opt_portfolio(i,mean_returns,cov_matrix):
    optimized_portfolio = hybrid_algorithm(n_solutions, n_iter, lb, ub, objective, alpha, beta0, gamma,mean_returns,cov_matrix)
    print("Portfolio allocation:", optimized_portfolio[0])
    portfolio_returns = np.sum(optimized_portfolio[0] * mean_returns)
    portfolio_volatility = np.sqrt(np.dot(optimized_portfolio[0].T, np.dot(cov_matrix,optimized_portfolio[0])))
    sharpe_ratio = portfolio_returns / portfolio_volatility
    sortino_ratio = portfolio_returns / np.sqrt(np.mean(np.minimum(returns_df, 0)**2))
    max_drawdown = np.max(np.maximum.accumulate((1+returns_df).cumprod()) - (1+returns_df).cumprod())
    print("Results for day: ",i)
    print("Portfolio allocation:\n",optimized_portfolio[0])
    print("Sharpe ratio:\n", sharpe_ratio)
    print("Sortino ratio:\n", sortino_ratio)
    print("Max drawdown:\n", max_drawdown)
    return(optimized_portfolio[0])

In [ ]:
def add(L):
  res = list()
  for j in range(0, len(L[0])):
    tmp = 0
    for i in range(0, len(L)):
        tmp = tmp + L[i][j]
    res.append(tmp)
  arr = np.array(res)
  result = arr/len(L[0])
  res = result.tolist()
  return(res)

In [ ]:
def calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov):
  cumulative_returns = 0
  cumulative_volatilities = 0

  for i in range(len(normalized_opt)):
      #print(i)
      # Calculate portfolio returns

      portfolio_returns = np.sum(normalized_opt[i] * interval_returns[i])
      #print(portfolio_returns)
      # Calculate portfolio volatility
      portfolio_volatility = np.sqrt(np.dot(normalized_opt[i].T, np.dot(interval_cov[i], normalized_opt[i])))
      #print(portfolio_volatility)
      # Add to cumulative returns and volatilities
      cumulative_returns += portfolio_returns
      cumulative_volatilities += portfolio_volatility

  sharpe_ratio = cumulative_returns / cumulative_volatilities
  print("Sharpe Ratio:", sharpe_ratio)
  return(sharpe_ratio)


In [ ]:
def calculate_roi(normalized_opt,ditv):
  i=0
  k=1
  capital=10000
  final_investment=[]
  final_amount=0
  invest = np.array([capital] * len(assets))
  investment = [x * y for x, y in zip(invest, normalized_opt[0])]
  remain=10000-sum(investment)
  for j in range(1,151):
    row_list = returns_df.loc[j, :].values.flatten().tolist()
    investment = [(x+(x * y)) for x, y in zip(investment, row_list)]
    i=i+1
    if(i==ditv and j!=150 ):
      print(investment)
      amount=sum(investment)+remain
      print("Investment at day ",j," is ",amount)
      cap = sum(investment)+remain
      investment = np.array([cap] * len(assets))
      investment = [x * y for x, y in zip(investment, normalized_opt[k])]
      print("New portfolio investment : ",investment)
      temp=sum(investment)
      print("Investment amount ",j," is ",amount)
      remain=cap-sum(investment)
      print("Remaning amount",remain)
      k=k+1
      i=0
    elif(j==150):
      print("Final Portfolio investment")
      final_investment=investment.copy()
      print(investment)
      amount=sum(investment)+remain
      final_amount=amount
      print("Investment at day ",j," is ",amount)

  n_periods = 252 / 150
  roi = ((final_amount / capital) - 1) * 100
  annualized_roi = (((1 + roi / 100) ** n_periods) - 1)*100
  print("Annualized ROI:", annualized_roi , "%")
  return(annualized_roi)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==5):
    temp_df=returns_df.iloc[j-4:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt",len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,5))

        PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
2  0.006646  0.018055 -0.004994 -0.005049  0.091762  0.007514 -0.005134   
3  0.009683  0.018818 -0.007247 -0.007777  0.097707  0.007716  0.003025   
4  0.007665  0.012438 -0.009241 -0.009711  0.073133  0.005877 -0.000371   
5  0.007187  0.008417 -0.009631 -0.010794  0.057347  0.004635  0.000343   
6  0.005267  0.002656 -0.010171 -0.011635  0.036429  0.002837 -0.002752   

        JPM       BAC      HSBC  
2  0.028507 -0.000543 -0.000154  
3  0.029644  0.021876  0.009885  
4  0.022066  0.012102  0.006415  
5  0.017417  0.013653  0.007042  
6  0.010648  0.004525  0.002903  
                PFE       JNJ         GOOGL      AAPL      COST       WMT  \
PFE    2.596079e-06  0.000008  9.907359e-07  0.000001  0.000031  0.000002   
JNJ    7.963952e-06  0.000046  1.225625e-05  0.000016  0.000170  0.000014   
GOOGL  9.907359e-07  0.000012  4.546702e-06  0.000006  0.000044  0.000004   
AAPL   1.408187e-06  0.000016  5.5833

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.06659042 0.09270457 0.01693305 0.01716844 0.53983029 0.0361811
 0.02764822 0.05602239 0.1110122  0.03594116]
Results for day:  5
Portfolio allocation:
 [0.06659042 0.09270457 0.01693305 0.01716844 0.53983029 0.0361811
 0.02764822 0.05602239 0.1110122  0.03594116]
Sharpe ratio:
 0.4193100340664129
Sortino ratio:
 PFE      0.476053
JNJ      0.469362
GOOGL    0.618350
AAPL     0.656273
COST     0.261783
WMT      0.761014
KR       0.890966
JPM      0.275140
BAC      0.210943
HSBC     0.618393
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
12  0.001284 -0.007724 -0.005857 -0.011483 -0.003568 -0.000261 -0.000580   
13  0.001579 -0.006439 -0.005227 -0.010925 -0.001945  0.000442  0.000528   
14  0.001577 -0.003473 -0.00

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.05560177 0.00310813 0.01449186 0.03320829 0.06423707 0.17366378
 0.10358114 0.07694944 0.18746232 0.28836041]
Results for day:  5
Portfolio allocation:
 [0.05560177 0.00310813 0.01449186 0.03320829 0.06423707 0.17366378
 0.10358114 0.07694944 0.18746232 0.28836041]
Sharpe ratio:
 -0.0037538614712465525
Sortino ratio:
 PFE     -0.002124
JNJ     -0.002094
GOOGL   -0.002759
AAPL    -0.002928
COST    -0.001168
WMT     -0.003396
KR      -0.003975
JPM     -0.001228
BAC     -0.000941
HSBC    -0.002759
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
17  0.003118  0.007113 -0.001228 -0.005359  0.033228  0.003487  0.009602   
18  0.003518  0.007143 -0.001670 -0.005412  0.031807  0.003833  0.008673   
19  0.004182  0.009732

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.0063587  0.24094523 0.02315646 0.04632844 0.20353665 0.40363097
 0.00710252 0.05249483 0.00109998 0.01096252]
Results for day:  5
Portfolio allocation:
 [0.0063587  0.24094523 0.02315646 0.04632844 0.20353665 0.40363097
 0.00710252 0.05249483 0.00109998 0.01096252]
Sharpe ratio:
 7.396834622095656
Sortino ratio:
 PFE      0.369794
JNJ      0.364597
GOOGL    0.480329
AAPL     0.509788
COST     0.203351
WMT      0.591150
KR       0.692095
JPM      0.213727
BAC      0.163859
HSBC     0.480363
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
147 -0.004021  0.002876  0.003502  0.000546  0.007492  0.002360 -0.000772   
148 -0.005866  0.002509  0.002322  0.000262  0.008200  0.001449 -0.001744   
149 -0.007748  0.001507 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [2.26528533e-03 2.04547528e-01 5.31667465e-02 5.35988364e-02
 4.85032885e-01 4.47335056e-02 2.00761425e-02 3.88652462e-02
 1.17675293e-04 1.02211869e-01]
Results for day:  5
Portfolio allocation:
 [2.26528533e-03 2.04547528e-01 5.31667465e-02 5.35988364e-02
 4.85032885e-01 4.47335056e-02 2.00761425e-02 3.88652462e-02
 1.17675293e-04 1.02211869e-01]
Sharpe ratio:
 3.3272629126274196
Sortino ratio:
 PFE      0.304586
JNJ      0.300305
GOOGL    0.395629
AAPL     0.419893
COST     0.167492
WMT      0.486908
KR       0.570053
JPM      0.176039
BAC      0.134964
HSBC     0.395657
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
Length of opt 30
[[0.364 0.047 0.06  0.011 0.153 0.04  0.05  0.129 0.063 0.073]
 [0.066 0.092 0.016 0.016 0.539 0.035 0.027 0.055 0.11  0.035]
 [0.055 0.002 0.013 0.032 0.0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==10):
    temp_df=returns_df.iloc[j-9:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt", len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,10))

         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
2   0.006646  0.018055 -0.004994 -0.005049  0.091762  0.007514 -0.005134   
3   0.009683  0.018818 -0.007247 -0.007777  0.097707  0.007716  0.003025   
4   0.007665  0.012438 -0.009241 -0.009711  0.073133  0.005877 -0.000371   
5   0.007187  0.008417 -0.009631 -0.010794  0.057347  0.004635  0.000343   
6   0.005267  0.002656 -0.010171 -0.011635  0.036429  0.002837 -0.002752   
7   0.004247  0.000023 -0.008946 -0.011212  0.025851  0.001602 -0.001911   
8   0.002394 -0.004605 -0.008891 -0.011675  0.010617  0.000061 -0.004307   
9   0.001759 -0.006825 -0.008151 -0.011874  0.002699 -0.000580 -0.003888   
10  0.000990 -0.008870 -0.007836 -0.012266 -0.004181 -0.001084 -0.004083   
11  0.000877 -0.009821 -0.007422 -0.012565 -0.007826 -0.000992 -0.003402   

         JPM       BAC      HSBC  
2   0.028507 -0.000543 -0.000154  
3   0.029644  0.021876  0.009885  
4   0.022066  0.012102  0.006415  
5   0.017417  0.013653 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.05453986 0.01877555 0.00113102 0.00810194 0.05108338 0.24062334
 0.35816295 0.06814097 0.10225647 0.09823451]
Results for day:  10
Portfolio allocation:
 [0.05453986 0.01877555 0.00113102 0.00810194 0.05108338 0.24062334
 0.35816295 0.06814097 0.10225647 0.09823451]
Sharpe ratio:
 1.003508610035528
Sortino ratio:
 PFE      0.661934
JNJ      0.652630
GOOGL    0.859792
AAPL     0.912523
COST     0.363999
WMT      1.058162
KR       1.238855
JPM      0.382572
BAC      0.293308
HSBC     0.859852
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
22  0.005329  0.013996  0.002160  0.000377  0.039615  0.005220  0.009950   
23  0.005670  0.013685  0.001995  0.000619  0.036887  0.005127  0.008873   
24  0.005762  0.012708  0.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [4.19526329e-01 1.28830844e-01 7.04347230e-02 1.25637998e-04
 0.00000000e+00 1.18686127e-01 0.00000000e+00 2.71711850e-01
 2.65019783e-04 9.78403455e-03]
Results for day:  10
Portfolio allocation:
 [4.19526329e-01 1.28830844e-01 7.04347230e-02 1.25637998e-04
 0.00000000e+00 1.18686127e-01 0.00000000e+00 2.71711850e-01
 2.65019783e-04 9.78403455e-03]
Sharpe ratio:
 5.9998164286969695
Sortino ratio:
 PFE      1.058735
JNJ      1.043854
GOOGL    1.375201
AAPL     1.459542
COST     0.582201
WMT      1.692484
KR       1.981496
JPM      0.611908
BAC      0.469133
HSBC     1.375298
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
32  0.005631  0.001664 -0.002709 -0.002279  0.001966  0.002188 -0.002849   
33  0.005051  0.001

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.06679828 0.07711906 0.         0.02267434 0.06875723 0.00324068
 0.6198722  0.         0.00163065 0.12154226]
Results for day:  10
Portfolio allocation:
 [0.06679828 0.07711906 0.         0.02267434 0.06875723 0.00324068
 0.6198722  0.         0.00163065 0.12154226]
Sharpe ratio:
 6.561017973663338
Sortino ratio:
 PFE      0.293414
JNJ      0.289290
GOOGL    0.381118
AAPL     0.404492
COST     0.161349
WMT      0.469048
KR       0.549144
JPM      0.169582
BAC      0.130014
HSBC     0.381145
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
142  0.002229  0.004476  0.004821 -0.000502  0.008065  0.005745  0.003061   
143  0.001575  0.004430  0.005807  0.000201  0.007858  0.005691  0.002542   
144  0.000600  0.004965

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.00384413 0.04179455 0.02609426 0.16770083 0.49201913 0.07289996
 0.05622006 0.13157446 0.00157765 0.01295062]
Results for day:  10
Portfolio allocation:
 [0.00384413 0.04179455 0.02609426 0.16770083 0.49201913 0.07289996
 0.05622006 0.13157446 0.00157765 0.01295062]
Sharpe ratio:
 3.3877545747222686
Sortino ratio:
 PFE      0.697251
JNJ      0.687451
GOOGL    0.905666
AAPL     0.961211
COST     0.383420
WMT      1.114619
KR       1.304953
JPM      0.402984
BAC      0.308957
HSBC     0.905730
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
Length of opt 15
[[0.016 0.    0.001 0.021 0.452 0.171 0.125 0.    0.024 0.183]
 [0.053 0.018 0.001 0.007 0.05  0.239 0.357 0.067 0.101 0.097]
 [0.411 0.125 0.068 0.    0.    0.115 0.    0.266 0.    0.009]
 [0.017 0.041 0.015 0.403 0.053 0.077 0.002 0.3

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==15):
    temp_df=returns_df.iloc[j-14:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt", len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,15))

         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
2   0.006646  0.018055 -0.004994 -0.005049  0.091762  0.007514 -0.005134   
3   0.009683  0.018818 -0.007247 -0.007777  0.097707  0.007716  0.003025   
4   0.007665  0.012438 -0.009241 -0.009711  0.073133  0.005877 -0.000371   
5   0.007187  0.008417 -0.009631 -0.010794  0.057347  0.004635  0.000343   
6   0.005267  0.002656 -0.010171 -0.011635  0.036429  0.002837 -0.002752   
7   0.004247  0.000023 -0.008946 -0.011212  0.025851  0.001602 -0.001911   
8   0.002394 -0.004605 -0.008891 -0.011675  0.010617  0.000061 -0.004307   
9   0.001759 -0.006825 -0.008151 -0.011874  0.002699 -0.000580 -0.003888   
10  0.000990 -0.008870 -0.007836 -0.012266 -0.004181 -0.001084 -0.004083   
11  0.000877 -0.009821 -0.007422 -0.012565 -0.007826 -0.000992 -0.003402   
12  0.001284 -0.007724 -0.005857 -0.011483 -0.003568 -0.000261 -0.000580   
13  0.001579 -0.006439 -0.005227 -0.010925 -0.001945  0.000442  0.000528   
14  0.001577

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.46335729 0.0072999  0.01029678 0.01549965 0.         0.13234487
 0.04953506 0.05706863 0.17567132 0.09041419]
Results for day:  15
Portfolio allocation:
 [0.46335729 0.0072999  0.01029678 0.01549965 0.         0.13234487
 0.04953506 0.05706863 0.17567132 0.09041419]
Sharpe ratio:
 7.31390242072374
Sortino ratio:
 PFE      1.659827
JNJ      1.636497
GOOGL    2.155965
AAPL     2.288191
COST     0.912743
WMT      2.653385
KR       3.106481
JPM      0.959316
BAC      0.735481
HSBC     2.156116
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
32  0.005631  0.001664 -0.002709 -0.002279  0.001966  0.002188 -0.002849   
33  0.005051  0.001420 -0.002948 -0.001938  0.001610  0.001754 -0.003055   
34  0.004363  0.000959 -0.0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.1516863  0.08590724 0.09937891 0.32635888 0.09684467 0.04470948
 0.00401754 0.20548808 0.00786295 0.00127348]
Results for day:  15
Portfolio allocation:
 [0.1516863  0.08590724 0.09937891 0.32635888 0.09684467 0.04470948
 0.00401754 0.20548808 0.00786295 0.00127348]
Sharpe ratio:
 9.555076775955053
Sortino ratio:
 PFE      0.344603
JNJ      0.339760
GOOGL    0.447609
AAPL     0.475061
COST     0.189498
WMT      0.550880
KR       0.644949
JPM      0.199168
BAC      0.152696
HSBC     0.447640
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
47 -0.003088  0.003464  0.004932  0.008114  0.002817 -0.000832  0.001401   
48 -0.003310  0.004406  0.006243  0.009110  0.003844 -0.000672  0.002291   
49 -0.003365  0.005049  0.

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.17154655 0.01598706 0.         0.00819907 0.34507732 0.15322914
 0.18952656 0.         0.02986573 0.06464572]
Results for day:  15
Portfolio allocation:
 [0.17154655 0.01598706 0.         0.00819907 0.34507732 0.15322914
 0.18952656 0.         0.02986573 0.06464572]
Sharpe ratio:
 6.984814858452911
Sortino ratio:
 PFE      0.616813
JNJ      0.608144
GOOGL    0.801185
AAPL     0.850321
COST     0.339187
WMT      0.986032
KR       1.154409
JPM      0.356494
BAC      0.273314
HSBC     0.801241
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
137  0.002859  0.002184 -0.001467 -0.003229  0.005325  0.002823  0.003146   
138  0.003023  0.002193 -0.000159 -0.002871  0.004520  0.003632  0.003102   
139  0.003134  0.002921

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.0254945  0.11223838 0.28289014 0.01350452 0.36867967 0.02708527
 0.08797041 0.02973346 0.         0.04492275]
Results for day:  15
Portfolio allocation:
 [0.0254945  0.11223838 0.28289014 0.01350452 0.36867967 0.02708527
 0.08797041 0.02973346 0.         0.04492275]
Sharpe ratio:
 3.5987643938054177
Sortino ratio:
 PFE      0.805188
JNJ      0.793871
GOOGL    1.045866
AAPL     1.110010
COST     0.442775
WMT      1.287167
KR       1.506965
JPM      0.465368
BAC      0.356785
HSBC     1.045940
dtype: float32
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306173
KR       0.248281
JPM      0.823145
BAC      1.156643
HSBC     0.372076
dtype: float32
Length of opt 10
[[0.425 0.007 0.    0.012 0.091 0.109 0.05  0.056 0.104 0.137]
 [0.462 0.006 0.009 0.014 0.    0.131 0.048 0.056 0.174 0.089]
 [0.147 0.083 0.096 0.318 0.094 0.043 0.003 0.2   0.007 0.001]
 [0.    0.    0.187 0.474 0.    0.279 0.005 0.0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
s_r

[1.8607853397707614, 1.63030256462093, 2.6926707786130537]

In [ ]:
roi

[334.1460899025599, 386.0160109746148, 266.7179570782305]

In [ ]:
returns_df.to_csv('lstm_test_returns.csv', index=False)